<a href="https://colab.research.google.com/github/anushayerram2025/AIML/blob/main/DomainClassification_of_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import pandas as pd

# Upload the Excel file
uploaded = files.upload()

# Get the uploaded file name
file_name = list(uploaded.keys())[0]

# Print the file name
print(f"Uploaded file: {file_name}")


Saving df_file.xlsx to df_file.xlsx
Uploaded file: df_file.xlsx


In [2]:

df = pd.read_excel(file_name,engine='openpyxl')

column1_values = df.iloc[:, 0].tolist()
column2_values = df.iloc[:, 1].tolist()

print("Paragraphs", len(column1_values))
print("labelss", len(column2_values))

Paragraphs 2225
labelss 2225


In [5]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


nltk.download('wordnet')


lemmatizer = WordNetLemmatizer()

stop_words = set(stopwords.words('english'))
print(stop_words)


tokenised_paragraphs = []
labels = []


for paragraph, label in zip(column1_values, column2_values):

    tokens = word_tokenize(paragraph.lower())


    cleaned_tokens = []
    for word in tokens:
        if word not in stop_words and word not in {'.',','}:
            cleaned_word = lemmatizer.lemmatize(word)
            cleaned_tokens.append(cleaned_word)


    tokenised_paragraphs.append(cleaned_tokens)
    labels.append(label)


print(tokenised_paragraphs[0])



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


{'ma', 'my', 'before', 'were', 'just', 'will', 'off', "you're", 'her', 'but', 'from', 'on', 'be', 'this', 'being', 'further', 'once', 'has', 'those', 'few', 'other', 'm', 'o', 'it', 'during', 'yours', 'd', 'after', 't', 'we', 'above', 'then', 'when', 'shan', "you'd", 'each', 'about', 'ours', "isn't", 'its', 'more', 'these', 'between', 'isn', 'yourself', 'in', 'had', 'an', 'doing', 'how', 'she', "aren't", "shouldn't", 'under', 'own', 'll', 'them', 'through', 'where', 'not', "needn't", 'than', 'won', 'that', 'very', "shan't", 'over', "don't", 'who', 'couldn', 'now', "should've", 'can', 'ourselves', 'all', "couldn't", 'why', 'mustn', 'having', 'so', 'up', "you'll", 'such', "you've", "doesn't", 'herself', 's', 'as', "she's", 'against', 'to', 'aren', 'haven', "it's", 'didn', 'should', 'wasn', 'doesn', 'ain', 'weren', 'hers', 'him', "weren't", 'do', 'some', 'is', 'at', 'only', 'both', 'are', 'below', 'yourselves', 'himself', 'our', "didn't", 'don', 'your', 'wouldn', "wasn't", 'again', 'whom'

In [7]:
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np




model = Word2Vec(min_count=1, window=1)
model.build_vocab(tokenised_paragraphs)
model.train(tokenised_paragraphs, total_examples=len(tokenised_paragraphs), epochs=10)

X = []
for paragraph in tokenised_paragraphs:
    vectors = []
    for word in paragraph:
        if word in model.wv.key_to_index:
            vectors.append(model.wv[word])
    if vectors:
        paragraph_vector = np.mean(vectors, axis=0)
        X.append(paragraph_vector)


y = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)
knn_classifier = KNeighborsClassifier(n_neighbors=3)
knn_classifier.fit(X_train, y_train)
knn_predictions = knn_classifier.predict(X_test)
knn_accuracy = accuracy_score(y_test, knn_predictions)
print("KNN Accuracy:", knn_accuracy * 100)



KNN Accuracy: 87.64044943820225
